In [1]:
import pandas as pd

In [2]:
with open('jolts.txt') as fp:
    jolts = sorted([ int(j) for j in fp.read().splitlines() ])

steps = pd.Series([0] + jolts + [jolts[-1] + 3]).diff().value_counts()
steps[1] * steps[3]

1848

In [3]:
jolt_set = set([0] + jolts + [jolts[-1] + 3])

data = pd.Series([0] + jolts + [jolts[-1] + 3], name='jolts').to_frame()
# it's legal to go one, two, or three jolts up
data['legal_jumps'] = data.jolts.apply(lambda s: (s+1, s+2, s+3))
# but those legal jumps might not be in my bag
data['jump_exists'] = data.legal_jumps.apply(lambda jumps: [j for j in jumps if j in jolt_set])

data['path_count'] = -1

# a way to look up the jolt value to the row in the data table
reverse_lookup = pd.Series(index = data.jolts.values, data = data.jolts.index )

In [4]:
# solved the last three by hand
data.loc[94, 'path_count']  = 0 
data.loc[93, 'path_count']  = 1 
data.loc[92, 'path_count']  = 1 

# Used those to build up the rest of the rows
for index in reversed(data.index):
    if data.loc[index, 'path_count'] == -1:
        path_count = 0
        for ex in data.loc[index, 'jump_exists']:
            reverse = reverse_lookup[ex]
            path_count += data.loc[reverse, 'path_count']
        data.loc[index, 'path_count'] = path_count
        
data.loc[0, 'path_count']

8099130339328